# Лабораторная работа №5
## Выполнил студент группы БФИ2302 Рыбаков А.Д. 

### Оглавление
1. [Задание 1](#Задание-№1)
2. [Задание 2](#Задание-№2)
4. [Вывод](#Вывод)

> Дополнительные модули, использованные при выполнение лабораторной

In [1]:
# Необходим при замере скорости выполнения кода
from datetime import datetime as t
# Нужен для создания словаря в алг. Бойера-Мура
from collections import defaultdict

### Задание №1
Реализовать методы поиска подстроки в строке. Добавить возможность ввода строки и подстроки с клавиатуры. Предусмотреть возможность существования пробела. Реализовать возможность выбора опции чувствительности или нечувствительности к регистру. Оценить время работы каждого алгоритма поиска и сравнить его со временем работы стандартной функции поиска, используемой в выбранном языке программирования.

#### Алгоритм Кнута-Морриса-Пратта

In [1]:
import time

# Реализация алгоритма Кнута-Морриса-Пратта
def kmp_search(text, pattern, case_sensitive=True):
    if not case_sensitive:
        text = text.lower()
        pattern = pattern.lower()

    # Функция для создания префикс-функции
    def compute_prefix_function(pattern):
        prefix = [0] * len(pattern)
        j = 0
        for i in range(1, len(pattern)):
            while j > 0 and pattern[i] != pattern[j]:
                j = prefix[j - 1]
            if pattern[i] == pattern[j]:
                j += 1
            prefix[i] = j
        return prefix

    prefix = compute_prefix_function(pattern)
    j = 0
    occurrences = []

    for i in range(len(text)):
        while j > 0 and text[i] != pattern[j]:
            j = prefix[j - 1]
        if text[i] == pattern[j]:
            j += 1
        if j == len(pattern):
            occurrences.append(i - (j - 1))
            j = prefix[j - 1]

    return occurrences

# Функция для ввода строки и подстроки
def input_string_and_substring():
    text = input("Введите строку: ")
    pattern = input("Введите подстроку: ")
    case_sensitive = input("Чувствительность к регистру (y/n): ").lower() == 'y'
    return text, pattern, case_sensitive

# Основная функция
def main():
    text, pattern, case_sensitive = input_string_and_substring()

    # Поиск с использованием алгоритма КМП
    start_time = time.time()
    kmp_result = kmp_search(text, pattern, case_sensitive)
    kmp_time = time.time() - start_time

    # Поиск с использованием стандартной функции Python
    start_time = time.time()
    if case_sensitive:
        standard_result = [i for i in range(len(text)) if text.startswith(pattern, i)]
    else:
        standard_result = [i for i in range(len(text)) if text.lower().startswith(pattern.lower(), i)]
    standard_time = time.time() - start_time

    # Вывод результатов
    print("\nРезультаты поиска:")
    print(f"Алгоритм КМП: {kmp_result}, время: {kmp_time:.6f} секунд")
    print(f"Стандартный поиск: {standard_result}, время: {standard_time:.6f} секунд")

if __name__ == "__main__":
    main()


Результаты поиска:
Алгоритм КМП: [0], время: 0.001006 секунд
Стандартный поиск: [0], время: 0.000000 секунд


#### Упрощенный алгоритм Бойера-Мура

In [2]:
def boyer_moore_simple(text, pattern):
    n = len(text)  
    m = len(pattern)  
    if m == 0:  
        return []

    # таблица сдвигов для неверно символа
    shift = {}  
    for i in range(m - 1):  
        shift[pattern[i]] = m - i - 1  

    # если символа нет в таблице, сдвигаем на всю длину подстроки
    default_shift = m

    # поиск подстроки
    i = 0  
    occurrences = []  
    while i <= n - m: 
        j = m - 1 
        while j >= 0 and text[i + j] == pattern[j]: 
            j -= 1
        if j == -1:
            occurrences.append(i)
            i += m
        else:
            
            char = text[i + j]
            if char in shift:  
                i += shift[char] 
            else: 
                i += default_shift 

    return occurrences  

text = "ABAAABCDABC"
pattern = "ABC"
result = boyer_moore_simple(text, pattern)
print("Найденные позиции:", result)

Найденные позиции: [4, 8]


### Задание №2
Написать программу, определяющую, является ли данное
расположение «решаемым», то есть можно ли из него за конечное число
шагов перейти к правильному. Если это возможно, то необходимо найти хотя
бы одно решение - последовательность движений, после которой числа будут
расположены в правильном порядке.
#### Входные данные: массив чисел, представляющий собой расстановку в
Порядке «слева направо, сверху вниз». Число 0 обозначает пустое поле.
Например, массив [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0] представляет
собой «решенную» позицию элементов.
#### Выходные данные: если решения нет, то функция должна вернуть
Пустой массив []. Если решение есть, то необходимо представить решение —
для каждого шага записывается номер передвигаемого на данном шаге
элемента. 

In [1]:
from collections import deque

# Функция для проверки, является ли головоломка решаемой
def is_solvable(puzzle):
    inversions = 0
    n = len(puzzle)
    for i in range(n):
        for j in range(i + 1, n):
            if puzzle[i] != 0 and puzzle[j] != 0 and puzzle[i] > puzzle[j]:
                inversions += 1
    # Для головоломки 4x4:
    # Если пустая клетка находится на чётной строке снизу, то количество инверсий должно быть нечётным.
    # Если пустая клетка находится на нечётной строке снизу, то количество инверсий должно быть чётным.
    empty_row = n // 4 - (puzzle.index(0) // 4)
    if empty_row % 2 == 0:
        return inversions % 2 != 0
    else:
        return inversions % 2 == 0

# Функция для поиска решения
def solve_puzzle(puzzle):
    if not is_solvable(puzzle):
        return []  # Если головоломка нерешаема, возвращаем пустой список

    target = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0]
    if puzzle == target:
        return []  # Если головоломка уже решена, возвращаем пустой список

    # Направления движения: вверх, вниз, влево, вправо
    directions = [(-1, 0, 'up'), (1, 0, 'down'), (0, -1, 'left'), (0, 1, 'right')]

    # Очередь для BFS
    queue = deque()
    queue.append((puzzle, []))  # текущее состояние, последовательность шагов

    # Множество для хранения посещённых состояний
    visited = set()
    visited.add(tuple(puzzle))

    while queue:
        current, path = queue.popleft()
        zero_index = current.index(0)
        zero_row, zero_col = zero_index // 4, zero_index % 4

        for dr, dc, move in directions:
            new_row, new_col = zero_row + dr, zero_col + dc
            if 0 <= new_row < 4 and 0 <= new_col < 4:
                new_index = new_row * 4 + new_col
                new_puzzle = current.copy()
                # Меняем местами 0 и соседний элемент
                new_puzzle[zero_index], new_puzzle[new_index] = new_puzzle[new_index], new_puzzle[zero_index]
                if tuple(new_puzzle) not in visited:
                    if new_puzzle == target:
                        return path + [new_puzzle[zero_index]]  # Возвращаем последовательность шагов
                    visited.add(tuple(new_puzzle))
                    queue.append((new_puzzle, path + [new_puzzle[zero_index]]))

    return []  # Если решение не найдено

# Ввод данных
puzzle = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 0, 15]  # Пример входных данных

# Решение головоломки
solution = solve_puzzle(puzzle)
if solution:
    print("Решение найдено! Последовательность шагов:", solution)
else:
    print("Решение не найдено.")

Решение найдено! Последовательность шагов: [15]


### Вывод